In [ ]:
import seaborn as sns
import pylab as rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib 
import pandas as pd 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional,GRU
from sklearn.preprocessing import MinMaxScaler
from  sklearn.model_selection import TimeSeriesSplit
from keras.callbacks import EarlyStopping
import tensorflow as tf
import keras_tuner as kt
import os
import warnings
from tcn import TCN

In [2]:
# %%
import seaborn as sns
import pylab as rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib 
import pandas as pd 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional,GRU
from sklearn.preprocessing import MinMaxScaler
from  sklearn.model_selection import TimeSeriesSplit
from keras.callbacks import EarlyStopping
import tensorflow as tf
import keras_tuner as kt
import os
import warnings
from tcn import TCN

# %% [markdown]
# 
arch='bitcn'
project_path='/home/j/usfq/Proyecto Integrador/StockPredictionModels/Data'
# %%
df=pd.read_csv(project_path+'/Data/Complete.csv')
df

# %%
#turn date into unix time
#df['Date'] = pd.to_datetime(df['Date'])
#df['Date'] = df['Date'].apply(lambda x: x.timestamp())
#df

# %%
#generate new dataframes for each ticker_symbol]
metric_labels=['Testing-MSE','Validation-MSE','testing-MAE','validation-MAE','testing-mape','validation-mape','testing-RMSE','validation-RMSE', 'testing-MPE','validation-MPE']
metrics_df=pd.DataFrame()
metrics_df['Metrics']=metric_labels
std_metrics_df=pd.DataFrame()
std_metrics_df['Metrics']=metric_labels
#save dataframe as csv
metrics_df.to_csv(project_path+f'/Results/{arch}_metrics.csv',index=False)
std_metrics_df.to_csv(project_path+f'/Results/{arch}_std_metrics.csv',index=False)
metrics=[]
df_dict={}
for key in df['ticker_symbol'].unique():
    df_dict[key]=df[df['ticker_symbol']==key]
    df_dict[key]=df_dict[key].drop(columns=['ticker_symbol'])
    df_dict[key]=df_dict[key].sort_values(by=['Date']).reset_index(drop=True)
    #df_dict[key]=df_dict[key].drop(columns=['Date'])
mse_t_p=[]
mae_t_p=[]
mape_t_p=[]
rmse_t_p=[]
mpe_t_p=[]
mse_v_p=[]
mae_v_p=[]
mape_v_p=[]
rmse_v_p=[]
mpe_v_p=[]

overall_mse_train=[]
overall_mse_val=[]
overall_std_train=[]
overall_std_val=[]

keys=df_dict.keys()
#to list
keys_list=list(keys)


# %%
ticker='TSLA'
print(f'Working on {ticker}...')
# %%
df=df_dict[ticker].copy()


# %%
#putting the close column on the last position
df=df[['Date', 'p_sentiment', 'Open', 'High', 'Low',
    'Volume', 'unrate', 'psr', 'm2', 'dspic', 'pce', 'reer', 'ir', 'ffer',
    'tcs', 'indpro', 'ccpi', 'Close']]

# %%
dates = pd.to_datetime(df['Date'])

# %%
cols=list(df)[1:]


# %%
df_for_training = df[cols].astype(float)

# %%
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_for_training)

# %%
#split scaled data into training, val and testing
#train_data=scaled_data[0:1000,:]
#val_data=scaled_data[1000:1125,:]
#test_data=scaled_data[1125:,:]

# %%
n_future = 1 # Number of days we want to predict into the future
n_past = 7 # Number of past days we want to use to predict the future

# %%
X=[]
y=[]
for i in range(n_past, len(scaled_data) - n_future +1):
    X.append(scaled_data[i - n_past:i, 0:df_for_training.shape[1]])
    y.append(scaled_data[i + n_future - 1:i + n_future, len(cols)-1])

# %%
#shape of X_s and y_s
X, y = np.array(X), np.array(y)

Working on TSLA...


In [6]:
print(X[1])

[[0.77183473 0.24678828 0.21948349 0.23172104 0.14147722 1.
  0.6        0.         0.         0.         0.         0.22902763
  0.00178319 0.         0.77924797 0.         0.23121105]
 [0.62544497 0.23144731 0.21127918 0.22138103 0.16859786 1.
  0.6        0.         0.         0.         0.         0.22902763
  0.00178319 0.         0.77924797 0.         0.23535347]
 [0.60139272 0.24268824 0.21334812 0.24090432 0.0686182  1.
  0.6        0.         0.         0.         0.         0.22902763
  0.00178319 0.         0.77924797 0.         0.23420473]
 [0.7053284  0.24084322 0.20985233 0.24171053 0.08301029 1.
  0.6        0.         0.         0.         0.         0.22902763
  0.00178319 0.         0.77924797 0.         0.23305598]
 [0.61846633 0.22755228 0.19622605 0.22400984 0.12022148 1.
  0.6        0.         0.         0.         0.         0.22902763
  0.00178319 0.         0.77924797 0.         0.21927104]
 [0.6895419  0.20749626 0.17657131 0.2039958  0.15913872 1.
  0.6     

In [5]:
print(y[0])

[0.21088176]
